# Logistic Regression Titanic


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [3]:
data = spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

# Select on specific columns you need

In [6]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

# drop columns with missing data

In [7]:
my_final_data = my_cols.na.drop()

### Working with Categorical Columns

In [8]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [9]:
# convert strings into numerical (Male==1, Female==2)
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')

# convert the numbers into one-hot encoding
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [11]:
assembler = VectorAssembler(inputCols=['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 
                                             'Parch', 'Fare'],
                                 outputCol='features')

# Model

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

## Pipelines 

tells how the stages of operation should go

In [15]:
from pyspark.ml import Pipeline

In [16]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

# Split data

In [17]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7, 0.3])

# Fit data

In [19]:
fit_model = pipeline.fit(train_titanic_data)

# test model on test data

In [20]:
results = fit_model.transform(test_titanic_data)

# Binary Evaluate

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [24]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [25]:
AUC = my_eval.evaluate(results)

In [27]:
AUC

0.7796653144016228